In [1]:
#  MODIFICATION HISTORY:
# Written in Python by Juan Pablo Velásquez (January 1st, 2021) 
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import matplotlib.dates as mdates
import h5py
from datetime import timedelta
import os
from model_utils import *

In [2]:
def getF107(month,year):
    dir_path = "geomagnetic_indices"
    filename = "hourly-geomagnetic-indices-%02d-%d.txt" % (month, year)
    filename = './' + dir_path + '/' + filename
    #current_month = 'Noviembre-2020'
    meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
    current_month = '%s-%d' % (meses[month-1], year)
    dir_plots = 'Plots-%s/' % current_month
    Data = pd.read_csv(filename,delimiter=r"\s+")#
    indices = GetIndices(Data)
    Data.set_index(indices)
    
    return Data

In [3]:
def getIndexF107(month, year, hour):
    F107 = getF107(month,year)
    #F107.tail()
    a = F107.loc[(F107['Hour'] == 2) & (F107['DayOfYear'] == 335), 'F107']
    return float(a)

In [4]:
'''
;  PURPOSE:
;    The VDRIFT_MODEL procedure is the main routine for obtaining vertical drifts. The vertical drifts
;    dependences of the local time and longitudinal (See details in Scherlies and Fejer, 1999). 
;  INPUTS:
;    DOY = A scalar of integer type giving the Day of year. (e.g. doy = 347)
;    HOUR = Set this keyword to an integer  from 00 to 23 to define the hour (00... 24hrs). By default 
;      hour =12.
;    F107CM = A scalar giving the solar flux F10.7cm. By default it will take 150.
;    LONGITUDE = Geographic Longitude.    
;  OUTPUTS:
;    vdrift= vertical drifts
'''

def drift_model(doy,f107cm,longitude):
    if doy == 0:
        doy = 161
    if longitude ==0:
        longitude = 270.0
    #constant = longitude
    start = 0.0
    finish = 24.0
    step = 0.5
    #nsteps = 49
    #step = int(finish-start)/nsteps
    ###############################################################
    st = 0.0
    end = 24.0
    nsteps = 49
    a = np.linspace(start,end,nsteps, endpoint=False)
    minutos = []
    horas = []
    for t in a:
        hour, minute = divmod(t, 1)
        minute *= 60
        result = '{}:{}'.format(int(hour), int(minute))
        #print(result)
        horas.append(int(hour))
        minutos.append(int(minute))
    ################################################################
    if (f107cm == 0):
        f107cm = 70
    ###################
    year = 2020
    d = datetime.datetime.strptime('{} {}'.format(doy, year),'%j %Y')
    ###################
    m = np.zeros(nsteps)
    steps = np.zeros(nsteps)
    #FOR i=start,(finish*(1+(profil EQ 0))) DO steps[i]=i*step
    j=0
    datetime_list = []
    for i in np.linspace(start,finish,nsteps,endpoint=False):
        steps[j]=i*step
        temp = i#start + i*step
        steps[j] = temp
        str_time = '%d-%02d-%02d %02d:%02d:%02d' % (year, d.month, d.day,horas[j], minutos[j],0)
        dt = datetime.datetime.strptime(str_time, '%Y-%m-%d %H:%M:%S')
        datetime_list.append(dt)
        
        j = j + 1
    time = steps
    yv = []
    #Calling VDrifts to compute vertical drifts.
    for ii in range(0,(nsteps-1) + 1):
        xt=time[ii]
        xl=longitude
        hout = int(xt)
        f107cm = getIndexF107(month, year, hour)
        y=vdrift(xt,xl,doy,f107cm)
        yv.append(y)
    return yv,datetime_list

### Comparison:

In [5]:
import datetime
format = "%Y-%m-%d %H:%M:%S"
year=2020
hour=10
#onth = 11
doy= 335
# = "2020-11-24 10:00:00" % (
d = datetime.datetime.strptime('{} {}'.format(doy, year),'%j %Y')
#t = datetime.datetime.strptime(s, format)
#t = dt.timetuple()
longitude=76.7
#oy = tt.tm_yday
f107cm = 80
y,xt = drift_model(doy,f107cm,longitude)

doys = [337,339,340,342,351,352,353,354,355,356,357,358,359,360,361,362]
days = [2,4,5,6,8,19,20,21,22,23,24,25,26,27]


NameError: name 'month' is not defined

In [ ]:
year = 2020
month = 12
day = 1
PlotFlag =  True
plot_format = 'png'
str_month = GetMonth(month)
current_month = '%s-%d' % (str_month, year)
directory = 'Data-%s/regular-files/' % current_month
str_format = 'png'
filename = 'jul%d%02d%02ddrifts.001.hdf5' % (year, month, day)

In [ ]:
file_hf5 = directory + filename
hf = h5py.File(file_hf5, 'r')
#with h5py.File(file_hf5, 'r') as f:
#    g = f.visit(print)

In [ ]:
def GetMatrix(directory, filename, PlotFlag, plot_format):
    ##########################################################
    ## 2020-06-16: Se verificó que la función trabaja
    ## correctamente. Se creara una nueva para hacer pruebas 
    ## con las dimensiones
    ##########################################################
    file_hf5 = directory + filename
    hf = h5py.File(file_hf5, 'r')
    #with h5py.File(file_hf5, 'r') as f:
    #    g = f.visit(print)
    '''
    days = np.array(hf['Data/Table Layout/']['day'],dtype=int)
    year = np.array(hf['Data/Table Layout/']['year'],dtype=int)
    month = np.array(hf['Data/Table Layout/']['month'],dtype=int)
    hour = np.array(hf['Data/Table Layout/']['hour'],dtype=int)
    minutes = np.array( hf['Data/Table Layout/']['min'],dtype=int)
    seconds = np.array(hf['Data/Table Layout/']['sec'],dtype=int)
    '''
    rango = hf['Data/Table Layout/']['gdalt']
    #rango2D = hf['Data/Array Layout/']['range']
    #Data/Array Layout/timestamps
    timestamps = hf['Data/Array Layout/']['timestamps']
    #snl =  hf['Data/Table Layout/']['snl']
    #snl2 = hf['Data/Array Layout/2D Parameters/snl']
      
    vipe1 = hf['Data/Array Layout/2D Parameters/vipe1'] 
    vipn1 = hf['Data/Array Layout/2D Parameters/vipn2']
    v_zonal = np.array(vipe1)
    v_vertical = np.array(vipn1)
    #snl2 = np.array(snl2)
    time_vector = []
    date_list = [] # list for datetime objects
  
    rango = getattr(rango, "tolist", lambda: rango)()
    ###########################################################
    ran_max = max(rango)
    ran_min = min(rango)
    #rang_list = list(rango)
    max_index = rango.index(ran_max)
    min_index = rango.index(ran_min)
    range_diff = np.diff(rango)
    delta_range = range_diff[0] #valor constante para todo el arreglo
    MinRange, MaxRange = np.min(rango), ran_max#np.max(rango)
    DataMatrixRows = int((MaxRange-MinRange)/delta_range) + 1
    #DataMatrix = np.ones((DataMatrixRows+1, snl2.shape[0]))*np.nan
    #RowInMatrix = np.array((rango-MinRange)/delta_range+1, dtype=int)
    range_array = np.linspace(MinRange, MaxRange, DataMatrixRows+1)
    #RangeMatrix = np.ones((DataMatrixRows+1, snl2.shape[0]))*np.nan
    #DataMatrix_v_zonal = np.ones((DataMatrixRows+1, snl2.shape[0]))*np.nan
    #DataMatrix_v_vertical = np.ones((DataMatrixRows+1, snl2.shape[0]))*np.nan
    
    #string_date = timestamps[0]#.strftime('%B %d, %Y, %r')
    #date_time_str = '2018-06-29 08:15:27.243860'
    
    datetime_objects = []
    for ts in timestamps:
        date_time_obj = datetime.datetime.fromtimestamp(ts)
        datetime_objects.append(date_time_obj)
    index = pd.DatetimeIndex(datetime_objects) #- timedelta(hours=5)

    #line.split()[0]
    #mes = #string_date.split()[0]
    mes = date_time_obj.month
    month_prime = 11
    if mes == 'June':
        mes ='Junio'
    if mes == 'July':
        mes = 'Julio'
    if mes == 'August':
        mes ='Agosto'
    if mes == 'September':
        mes ='Septiembre'
    if mes == 'October':
        mes = 'Octubre'
    if mes == 'November':
        mes ='Noviembre'

    dia = date_time_obj.day
    anio = date_time_obj.year
    
    dir_plots = '.'#'Plots-150km-%s-%d' % (mes, anio)
 
    #######################################################################################################
    if (PlotFlag):
        fig, ax = plt.subplots(figsize=(12, 6))
        #plt.rcParams['xtick.labelsize']=14
        
        plt.style.use('dark_background')
        x_min = mdates.date2num(np.min(index))
        x_max = mdates.date2num(np.max(index))
        #extent=[x_min, x_max,ran_min,ran_max]
        print("Shapes: ", v_vertical.T.shape, range_array.shape, len(datetime_objects))
        #ax = plt.imshow(v_vertical.T, cmap='jet',aspect='auto',interpolation='nearest',origin="lower", extent=extent)
        clrs= ax.pcolormesh(mdates.date2num(datetime_objects), range_array, v_vertical.T, cmap='jet')
        ax.xaxis_date()
        date_format = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(date_format)
        ax.set_xlabel("Hora Local (h)", fontsize=16)
        ax.set_ylabel("Rango (km)", fontsize=17)
        fig_title = r'Derivas Verticales ISR (%d-%02d-%02d)' % (anio, month_prime, dia) 
        plt.title(fig_title, fontsize=15)
        str_date = '(%d-%02d-%02d)' % (anio, month_prime, dia)
        ax.set_ylim(300, 1200)
        # This simply sets the x-axis data to diagonal so it fits better.
        fig.autofmt_xdate()
        box=ax.get_position()
        cbarax=fig.add_axes([box.x0+box.width+0.01, box.y0, 0.025, box.height])
        cb=plt.colorbar(clrs,cax=cbarax)
        cb.set_label(r'Derivas verticales (m/s)')
        #cb2 = fig.colorbar(im2)
        #cb2.set_label(r'$lo      '''y,xt
       
        #plt.rcParams['xtick.labelsize']=14
      
        #plt.savefig(r'%s/ecos-150km-%d-%02d-%02d-v-zonal.%s' % (dir_plots,anio, month_prime, dia, plot_format))
        #plt.savefig(r'%s/ecos-150km-%d-%02d-%02d-SNR.png' % (dir_plots,anio, month_prime, dia))
        plt.show()
        plt.close(fig)   
        
    #####################################################################################
  
    return index, range_array, rango, dir_plots, v_zonal, v_vertical, timestamps, str_date

In [ ]:
#index, range_array, rango, dir_plots, v_zonal, v_vertical, timestamps, str_date = GetMatrix(directory, filename, PlotFlag, plot_format)
str_format = 'png'
index, range_array, rango, dir_plots, v_zonal, v_vertical, timestamps, str_date = GetMatrix(directory, filename, PlotFlag, plot_format)
print(index)
#print("Data, index, range_array y len(rango): ")
#print(data.shape, index.shape, range_array.shape, len(rango))
#str_date = '%d-%02d-%02d' % (index[0].year,index[0].month,index[0].day)
print(str_date)
prev_times = [' ']
prev_stamps = [' ']

#print(time_vector)
k = 0
#################################
datetime_objects = []
for ts in timestamps:
    if ts in prev_stamps:
        print('Same timestap')
    else:
        date_time_obj = datetime.datetime.fromtimestamp(ts)
        datetime_objects.append(date_time_obj)
new_index = pd.DatetimeIndex(datetime_objects) # timedelta(hours=5)
#################################

#'''
data = v_vertical

#fig, ax = plt.subplots(figsize=(12, 6))       
v_vert_avg = np.nanmean(data,axis=1)
v_vert_std = np.nanstd(data,axis=1)
print("Vertical Average shape: ", v_vert_avg.shape, new_index.shape, new_index[0], new_index[-1])

        #df_std = pd.Series(v_vert_std, index=new_index)
#'''

In [ ]:
#v_vert_avg = np.nanmean(v_vertical,axis=1)
#v_vert_std = np.nanstd(v_vertical,axis=1)

df = pd.Series(v_vert_avg, index=index)
df_std = pd.Series(v_vert_std, index=index)

In [ ]:
fig, axs = plt.subplots(figsize=(12, 6))

df.groupby(df.index.hour).mean().plot(yerr=df.groupby(df_std.index.hour).std(),rot=0,ax=axs)
fig_title = 'Derivas verticales de plasma %d-%02d-%02d' % (index[0].year,index[0].month, index[0].day)
axs.set_title(fig_title, fontsize=16)
#axs.set_facecolor("white")
date_format = mdates.DateFormatter('%H:%M')
#axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
axs.set_xlabel('Hora Local (h)', fontsize=18)
axs.set_ylabel('m/s', fontsize=18)
#axs.set_xlim([100,400])
axs.xaxis_date()
axs.xaxis.set_major_formatter(date_format)
#plt.savefig('promedio-mensual-drifts-agosto-2020.pdf')
#'''#plt.plot(dt_index, df.groupby(index,df.index.hour).mean())


### Promedio mensual:

In [ ]:
PlotFlag = True
plot_format= 'png'
h_min = []
h_max = []
frames_avg = []
frames_std = []
num_stamps = [] 
t_h_min = []
t_h_max = []
for filename2 in os.listdir(directory):
    if filename2.endswith(".hdf5"):
        print(filename2)
        
        index, range_array, rango, dir_plots, v_zonal, v_vertical, timestamps, str_date = GetMatrix(directory, filename2, PlotFlag, plot_format)
        print("Data, index, range_array y len(rango): ")
        #print(data.shape, index.shape, range_array.shape, len(rango))
        #str_date = '%d-%02d-%02d' % (index[0].year,index[0].month,index[0].day)
        
        print(str_date)
        prev_times = [' ']
        k = 0
        for time in index:
            if prev_times[k] == time:
                   print('Same Time')
            else:
                #print 'Different time'
                if not time in prev_times:
                    prev_times.append(time)
                prev_time = time
        
 
    ###########################################################
        ran_max = max(rango)
        ran_min = min(rango)
        max_index = rango.index(ran_max)
        min_index = rango.index(ran_min)
    ############################################################
        #num_stamps.append(len(timestamps))
        prev_times = prev_times[1:]
        num_diff_times = len(prev_times)
        print(num_diff_times, v_vertical.shape)
       
        new_index = pd.DatetimeIndex(prev_times)
        #print("Vertical Average shape: ", v_vert_avg.shape, new_index.shape, new_index[0], new_index[-1])
        #fig, ax = plt.subplots(figsize=(12, 6))
       
        str_format = 'png'
        v_vert_avg = np.nanmean(v_vertical,axis=1)
        v_vert_std = np.nanstd(v_vertical,axis=1)
        
        df_std = pd.Series(v_vert_std, index=new_index)
        print(v_vert_avg.shape, index.shape)
        df = pd.Series(v_vert_avg, index=new_index)
        #df.set_index(index, inplace=True, drop=True)
        str_date = '2020-%02d-%02d %02d:%02d:00' % (new_index[0].month, new_index[0].day, new_index[0].hour, new_index[0].minute)
        string_t0 = '2020-%02d-%02d %02d:%02d:00' % (new_index[0].month, new_index[0].day, 0,0)
        string_tf = '2020-%02d-%02d %02d:00:00' % (new_index[0].month, new_index[0].day, 23)
        '''
        ax = df[string_t0:string_tf].plot()
        ax.set_ylabel('(m/s)', fontsize=20)
        ax.set_title('Velocidad Vertical Promedio (m/s) %s' % str_date, fontsize=20)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H %M'))
        #plt.savefig('velocidad-vertical-promedio-%s.%s' % (str_date, str_format))
        #plt.show()
        #plt.close(fig)
        print("Range array shape: ", np.array(rango).shape)
        
        fig1, ax1 = plt.subplots(figsize=(12, 6))
        #str_date = '%d-%02d-%02d' % (new_index[0].year,new_index[0].month,new_index[0].day)
        #str_format = 'png'
        #v_vert_avg = np.nanmean(v_vertical,axis=1)
        #v_vert_std = np.nanstd(v_vertical,axis=1)
        '''
        df_std = pd.Series(v_vert_std, index=new_index)
        #df = pd.Series(v_vert_avg, index=new_index)
        #df.set_index(index, inplace=True, drop=True)
        '''
        ax1 = df_std.plot()#[string_t0:string_tf].plot()
        ax1.set_ylabel('(m/s)', fontsize=20)
        ax1.set_title(r'Velocidad Vertical $\sigma$ (m/s) %s' % str_date, fontsize=20)
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H %M'))
        '''
        #plt.savefig('velocidad-vertical-desviacion-estandar-%s.%s' % (str_date, str_format))
        #plt.show()
        #plt.close(fig)
        frames_avg.append(df)#string_t0:string_tf])
        frames_std.append(df_std)#_std[string_t0:string_tf])
        #'''
#'''

In [ ]:
list_avg = []
list_std = []

In [ ]:

series_avg = pd.concat(frames_avg)
series_std = pd.concat(frames_std)
fig, axs = plt.subplots(figsize=(12, 6))
plt.rcParams['ytick.labelsize']=16
plt.rcParams['xtick.labelsize']=16
#plt.style.use('seaborn-white')#'ggplot')#'classic')
series_avg.groupby(series_avg.index.hour).mean().plot(yerr=series_avg.groupby(series_std.index.hour).std(),rot=0,ax=axs)
fig_title = 'Derivas verticales - ISR (promedio mensual) 2020-%s' % ('noviembre')
axs.set_title(fig_title, fontsize=16)
#axs.set_facecolor("white")
list_avg.append(series_avg.groupby(series_avg.index.hour).mean())
list_std.append(series_avg.groupby(series_avg.index.hour).std())
#axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
axs.set_xlabel('Hora Local (h)', fontsize=18)
axs.set_ylabel('m/s', fontsize=18)
#axs.set_xlim([100,400])

plt.savefig('promedio-mensual-drifts-%s.png' % current_month)

In [ ]:
import matplotlib.pyplot as plt
fig, ax1 = plt.subplots(figsize=(20, 9))
#plt.rcParams['xtick.labelsize']=14
date_format = mdates.DateFormatter('%H:%M')

plt.style.use('dark_background')
#df.index = pd.to_pydatetime(df.index)
#df.index.to_pydatetime()
#stri = '%d-%02d-%02d %02d:%02d:%02d' % (xt[0].year, xt[0].month, xt[0].day, 2,0,0)
#strf = '%d-%02d-%02d %02d:%02d:%02d' % (xt[0].year, xt[0].month, xt[0].day, 20,0,0)
#dfp = df.loc[stri:strf]#plt.plot(dt_index, df.groupby(index,df.index.hour).mean())

#dfp_std = df_std.loc[stri:strf]
#dt_index = index.to_pydatetime()
df.groupby(df.index.hour).mean().plot(yerr=df.groupby(df_std.index.hour).std(),rot=0)
list_avg.append(df.groupby(df.index.hour).mean())
list_std.append(df.groupby(df_std.index.hour).std())
#ax1.plot(xt,y, label='Data ISR')
#plt.plot(dt_index, df.groupby(index,df.index.hour).mean())
#ax1.xaxis_date()
#ax1.xaxis.set_major_formatter(date_format)
ax1.set_xlabel("Hora Local (h)", fontsize=22)
ax1.set_ylabel("Ecos de 150 km - Derivas verticales (m/s)", fontsize=22)
ax1.set_ylim(-30, 30)
str_date = '%d-%02d-%02d' % (xt[0].year, xt[0].month, xt[0].day)
plt.title('Drifts Verticales - Modelo de Scherliess-Fejer - %s' % str_date, fontsize = 24)
plt.rcParams['ytick.labelsize']=22
plt.rcParams['xtick.labelsize']=22
plt.tight_layout()

#plt.savefig('modelo-scherliess-fejer-derivas-verticales-%s.png' % str_date)
#plt.plot(dt_index, df.groupby(index,df.index.hour).mean())

#tt.tm_yday
#print(xt)

### Promedio mensual - Modelo de Scherliess-Fejer

In [ ]:
doys = [337,339,340,342,351,352,353,354,355,356,357,358,359,360,361,362]
days = [2,4,5,6,8,19,20,21,22,23,24,25,26,27]
model_pred = []


In [ ]:
month = 12
year = 2020
hour = 10
doy = 336
#a = F107.loc[(F107['Hour'] == 2) & (F107['DayOfYear'] == 335), 'F107']
#print(a)
#df.loc[(df['Salary_in_1000']>=100) & (df['Age']< 60) & (df['FT_Team'].str.startswith('S')),['Name','FT_Team']]
solar_ind = getIndexF107(month, year, hour)
print(solar_ind)

In [ ]:
format = "%Y-%m-%d %H:%M:%S"
year=2020
hour=10
longitude=76.7
#month = 11
#year = 2020
#F107 = getF107(month,year)
#F107.tail(20)
#month = 11
#doy= 329
# = "2020-11-24 10:00:00" % (
modDrifts = []
xtimes = []
for doy in doys:
    d = datetime.datetime.strptime('{} {}'.format(doy, year),'%j %Y')
#t = datetime.datetime.strptime(s, format)
#t = dt.timetuple()
#oy = tt.tm_yday
#f107cm = 80
    f107cm = getIndexF107(month, year, hour)
    y,xt = drift_model(doy,f107cm,longitude)
    df_v = pd.Series(y, index=xt)
    #mod_avg = np.nanmean(df_v)
    modDrifts.append(df_v)
    #mod_std = np.nanstd(v_vertical)
    #modDrifts_std.append(mod_std)    
    xtimes.append(xt)

In [ ]:
#'''
mod_series_avg = pd.concat(modDrifts)
#mod_series_std = pd.concat(modDrifts_std)
list_avg.append(mod_series_avg.groupby(mod_series_avg.index.hour).mean())
list_std.append(mod_series_avg.groupby(mod_series_avg.index.hour).std())
fig, axs = plt.subplots(figsize=(12, 6))
#plt.rcParams['ytick.labelsize']=16
#plt.rcParams['xtick.labelsize']=16
#plt.style.use('seaborn-white')#'ggplot')#'classic')
mod_series_avg.groupby(mod_series_avg.index.hour).mean().plot(yerr=mod_series_avg.groupby(mod_series_avg.index.hour).std(),rot=0,ax=axs)
fig_title = 'Derivas verticales - Modelo de Scherliess-Fejer' 
axs.set_title(fig_title, fontsize=16)
#axs.set_facecolor("white")

#axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
#axs.set_xlabel('Hora Local (h)', fontsize=18)
#axs.set_ylabel('m/s', fontsize=18)
#'''
plt.savefig('Scherliess-Fejer-%s.png' % current_month)

In [ ]:
fig3, ax3 = plt.subplots(figsize=(22,10))
exp = ['Datos ISR', 'Modelo de Scherliess-Fejer']
markers = ['o', 'v']
plt.rcParams['ytick.labelsize']=20
plt.rcParams['xtick.labelsize']=20
for i in range(0,2):
    #for i in range(len(stations)):
    ax3 = list_avg[i].plot(yerr=list_std[i],label=exp[i], marker=markers[i])
    ax3.set_xlabel('Hora Local(h)', fontsize=25)
    ax3.set_ylabel(r'Derivas verticales (m/s) ', fontsize=25)
    ax3.set_title('Promedio mensual - Derivas verticales vs. modelo de Scherliess-Fejer', fontsize=25)
    #plt.plot(daily_averages, color=colors[i], label=stations[i])
    ax3.tick_params(axis='y', labelsize=24)
    ax3.tick_params(axis='x', labelsize=24)
    #plt.grid(True)
plt.legend(fontsize=23)    
print(current_month)
plt.savefig('comparison-vdrifts-data-ISR-vs-Scherliess-Fejer-%s.pdf' % current_month)

### Calculando RTI Promedio:

In [ ]:
year = 2020
month = 11
days = [24,25,26,27] 
PlotFlag =  True
plot_format = 'png'
filenames = []
for dia in days:
    fname = 'jro%d%02d%02ddrifts.001.hdf5' % (year, month, dia)
    print(fname)
    filenames.append(fname)


In [ ]:
def GetStandardDriftMatrix(directory,filename, plot_format):
    file_hf5 = directory + filename
    hf = h5py.File(file_hf5, 'r')
    alturas = hf['Data/Table Layout/']['gdalt']
    range_file = np.array(hf['Data/Array Layout/']['range'])
    timestamps = hf['Data/Array Layout/']['timestamps']
    vipe1 = hf['Data/Array Layout/2D Parameters/vipe1'] 
    vipn1 = hf['Data/Array Layout/2D Parameters/vipn2']
    rango2D = np.array(hf['Data/Array Layout/2D Parameters/range'])
    v_zonal = np.array(vipe1)
    v_vertical = np.array(vipn1)

    alturas = np.array(getattr(alturas, "tolist", lambda: alturas)())

    ###########################################################
    ## Arreglos del archivo:
    ran_max = np.max(rango2D)
    ran_min = np.min(rango2D)
    range_diff = np.diff(rango2D,axis=1)
    delta_range = range_diff[0][0]
    ranNum = int((ran_max-ran_min)/delta_range) + 1
    range_file = np.arange(ran_min,ran_max+delta_range,delta_range)#np.linspace(ran_min, ran_max, ranNum)
    #delta_range = range_diff[0] #valor constante para todo el arreglo
    print("ran_max: ", ran_max)
    
    datetime_objects = []
    for ts in timestamps:
        date_time_obj = datetime.datetime.fromtimestamp(ts)
        datetime_objects.append(date_time_obj)
    index = pd.DatetimeIndex(datetime_objects) #- timedelta(hours=5)
    ###Creando arreglo de datetime objetcs:
    strf = '%d-%02d-%02d 23:55:00' % (index[0].year, index[0].month, index[0].day)
    str0 = '%d-%02d-%02d 00:00:00' % (index[0].year, index[0].month, index[0].day)
    dt0 = datetime.datetime.strptime(str0, '%Y-%m-%d %H:%M:%S')
    dtf = datetime.datetime.strptime(strf, '%Y-%m-%d %H:%M:%S')
    ntimes = 288
    dt_list = []
    dt_array = np.arange(dt0,dtf+timedelta(seconds=300),timedelta(seconds=300))#np.array(dt_list)
    delta_time_array = np.diff(index)[0]/ np.timedelta64(1, 's')#(dtf-dt0).total_seconds()/ntimes#/np.timedelta64(1, 's')#np.linspace(dt0,dtf,timedelta(seconds=300))
    print("delta_time_array: ", delta_time_array)
    ###################################################################
    ## Arreglos Estandarizados##########################
    MinRange, MaxRange = 0, 1425.0#ran_min, ran_max#np.max(rango)
    DataMatrixRows = int((MaxRange-MinRange)/delta_range) + 1 #
    DataMatrix = np.ones((DataMatrixRows, ntimes))*np.nan
    RowInMatrix = np.array((rango2D[1,:]-MinRange)/delta_range, dtype=int)
    range_array = np.arange(MinRange,MaxRange+delta_range,delta_range)
    ###################################################################
    col = 0 #counter for current columns
    PastRow = 0 #saving past row index
    mes = date_time_obj.month
    month_prime = 11
    mes = GetMonth(month_prime)
    dia = date_time_obj.day
    anio = date_time_obj.year

    dir_plots = '.'#'Plots-150km-%s-%d' % (mes, anio)
    print("Shapes: ", v_zonal.T.shape, rango2D.shape)
    print("Shapes: ", v_vertical.T.shape, range_array.shape, len(datetime_objects))
    print("Shapes: ", DataMatrix.shape, range_file.shape, dt_array.shape)
    #######################################################################################################
    print(range_array[-1])
    print(range_file[-1])
    #print(rango2D[-1])
    #print(rango[-1])
    diff = list(np.diff(rango2D).flatten())
    print(diff.count(14), diff.count(15), len(diff))
    
    datetime_objects = np.array(datetime_objects)
    RowInMatrixTime = np.array((datetime_objects-dt0)/timedelta(seconds=300),dtype=int)
    DataMat = np.ones((ntimes,DataMatrixRows))*np.nan
    col = 0 #counter for current columns
    for i in range(0,datetime_objects.shape[0]):
        row = RowInMatrixTime[i]
        DataMat[row,:] = v_vertical[i,:]
    
    if (PlotFlag):
        fig, ax = plt.subplots(figsize=(12, 6))
    #plt.rcParams['xtick.labelsize']=14
        plt.style.use('dark_background')
        clrs= ax.pcolormesh(mdates.date2num(dt_array), range_array, DataMat.T, cmap='jet')
        ax.xaxis_date()
        date_format = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(date_format)
        ax.set_xlabel("Hora Local (h)", fontsize=16)
        ax.set_ylabel("Rango (km)", fontsize=17)
        fig_title = r'Derivas Verticales ISR (%d-%02d-%02d)' % (anio, month_prime, dia) 
        plt.title(fig_title, fontsize=15)
        str_date = '(%d-%02d-%02d)' % (anio, month_prime, dia)
        #ax.set_ylim(300, 1200)
        # This simply sets the x-axis data to diagonal so it fits better.
        fig.autofmt_xdate()
        box=ax.get_position()
        cbarax=fig.add_axes([box.x0+box.width+0.01, box.y0, 0.025, box.height])
        cb=plt.colorbar(clrs,cax=cbarax)
        cb.set_label(r'Derivas verticales (m/s)')
        plt.savefig(r'%s/Drifts-ISR-%d-%02d-%02d-v-zonal.%s' % (dir_plots,anio, month_prime, dia, plot_format))
        plt.show()
        plt.close(fig)   
    #####################################################################################
    
    
    return DataMat.T, range_array, dt_array

In [ ]:
auxMat = np.zeros((96,288))
for filename in filenames:
    data, range_array, dt_array = GetStandardDriftMatrix(directory,filename, plot_format)
    data[np.isnan(data)]=0.0
    auxMat = auxMat + data
avgMat = auxMat/len(filenames)


In [ ]:
avgMat[avgMat==0]=np.nan
fig, ax1 = plt.subplots(figsize=(12, 6))
    #plt.rcParams['xtick.labelsize']=14
plt.style.use('dark_background')
clrs= ax1.pcolormesh(mdates.date2num(dt_array), range_array, avgMat, cmap='jet')
ax1.xaxis_date()
date_format = mdates.DateFormatter('%H:%M')
ax1.xaxis.set_major_formatter(date_format)
ax1.set_xlabel("Hora Local (h)", fontsize=16)
ax1.set_ylabel("Rango (km)", fontsize=17)
fig_title = r'Promedio Mensual Derivas Verticales ISR'  
plt.title(fig_title, fontsize=15)
#str_date = '(%d-%02d-%02d)' % (anio, month_prime, dia)
ax1.set_ylim(300, 600)
# This simply sets the x-axis data to diagonal so it fits better.
fig.autofmt_xdate()
box=ax1.get_position()
#cmap=plt.cm.RdBu
cbarax=fig.add_axes([box.x0+box.width+0.01, box.y0, 0.025, box.height])
cb=plt.colorbar(clrs,cax=cbarax)
cb.set_label(r'Derivas verticales (m/s)', fontsize=16)
#plt.clim(-30,30)
cb.mappable.set_clim(-30,30)
plt.savefig(r'Promedio-Mensual-Derivas-Verticales-%s.png' % current_month)

In [ ]:
fig = plt.figure(figsize=(30,10))
plt.rcParams['ytick.labelsize']=29
plt.rcParams['xtick.labelsize']=29
#fig=plt.figure()
ax=fig.add_subplot(111, label="1")
ax2=fig.add_subplot(111, label="2", frame_on=False)
plt.title("Promedio de los días de mediciones - derivas verticales (m/s)", fontsize=30)

ax.plt= ax.pcolormesh(dt_array, range_array, data, cmap=plt.cm.RdBu_r)
color='red'
color='white'
ax.set_xlabel('Hora Local',fontsize=29)
ax.set_ylabel('Rango (km)',fontsize=29)
ax.set_ylim(300, 400)
ax.xaxis_date()
date_format = mdates.DateFormatter('%H:%M')
ax.xaxis.set_major_formatter(date_format)
#ax.set_xlabel("x label 1", color="C0")
#ax.set_ylabel("y label 1", color="C0")
#ax.set_ylim(130,180)
#xticks=numpy.array([0,43200,86400])
#xticks=numpy.arange(0,86400,3)
#xticks=numpy.linspace(0,86400,13,endpoint=True)
#ax.set_xticks(xticks)
#xticks_label=['00:00','02:00','04:00','06:00','08:00','10:00','12:00','14:00','16:00','18:00','20:00','22:00','24:00']
#ax.set_xticklabels(xticks_label)
#ax.set_xlim(25200,57600)
#cbar.set_label(r'$log_{10}SNR$',fontsize=30)
cbar=fig.colorbar(ax.plt,ax=ax,pad=0.08)
cbar.set_label(r'Derivas verticales (m/s) ', fontsize=26)
cbar.ax.tick_params(labelsize=28)
#plt.title(r'Ecos de 150 km (2020-11-08)',fontsize=33)
#ax.tick_params(axis='x', colors="C0")
#ax.tick_params(axis='y', colors="C0")
#cbar.remove()




#xraycolor='tab:blue'
#ax2.plot(xrs_a_series.loc["2020-11-08T 07:00:00":"2020-11-08T 16:00:00"],linestyle='-', color=xraycolor,linewidth=4,label='Flujo de Rayos X')
#ax2 = list_avg[1].plot(yerr=list_std[1],label=exp[1], marker=markers[1])
list_avg[1].plot(yerr=list_std[1],label=exp[1], marker=markers[1],color='black')
list_avg[0].plot(yerr=list_std[0],label=exp[0], marker=markers[1],color='red')

plt.legend(loc="upper right",prop={'size':25})
#ax2.xaxis.tick_top()
ax2.get_xaxis().set_visible(False)
ax2.yaxis.tick_right()
#ax2.set_xlabel('x label 2', color="C1") 
ax2.set_ylabel('y label 2')#, color="C1")       
#ax2.xaxis.set_label_position('top') 
ax2.yaxis.set_label_position('right') 
#ax2.tick_params(axis='x')#, colors="C1")
ax2.tick_params(axis='y')#, colors="C1")
#ax2.set_xlim("2020-11-08 07:00:00","2020-11-08 16:00:00")
ax2.set_ylabel(r'Derivas verticales (m/s) ', fontsize=26)
cbar2=fig.colorbar(ax.plt,ax=ax2,pad=0.08)
cbar2.mappable.set_clim(-30,30)

cbar2.remove()

plt.savefig('RTI-promedio-vs-promedio-derivas-verticales-ISR-data-modelo-%s.png' % current_month)
plt.show()